<a href="https://colab.research.google.com/github/samar018/BlockClassification-using-food-habits-and-clinical-data/blob/main/DecisiontreeUsingsplitingdataand_ROC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.tree import DecisionTreeClassifier
from scipy import interp

import itertools as it
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection  import cross_val_score, cross_val_predict,train_test_split
from sklearn.impute import SimpleImputer


from sklearn.metrics import roc_curve, auc,classification_report, confusion_matrix,accuracy_score


In [ ]:

# Import some data to play with

dataset = pd.read_csv('/home/god/Downloads/New folder (3)/Final Documnets/Final Documnets/Code/DataSet/datafinal.csv')
X = dataset.iloc[:, 1:-2].values
y = dataset.iloc[:,26].values

print(X.shape)
print(y.shape)
print(X.shape[0] != y.shape[0])
#X=np.matrix(dataset.iloc[:, 1:-2].values)
#
#y=np.matrix(dataset.iloc[:, 1:-2].values)

simpleImputer = SimpleImputer(missing_values = np.nan, strategy = 'median')
simpleImputer= simpleImputer.fit(X[:, 1:-1])
X[:, 1:-1] = simpleImputer.transform(X[:, 1:-1]

In [ ]:

y = label_binarize(y, classes=[0, 1, 2,3,4,5])
n_classes = y.shape[1]

In [ ]:

# Add noisy features to make the problem harder
random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

In [ ]:
classifier = ( DecisionTreeClassifier(criterion='gini', 
     splitter='best', max_depth=3, min_samples_split=60, min_samples_leaf=1,
     max_features=None, random_state=random_state ,max_leaf_nodes=5,class_weight=None))


In [ ]:
"""using spliting """
#

# shuffle and split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, random_state=random_state)

In [ ]:

#x_train,X_test,y_train,y_test=train_test_split(np.transpose(X),np.transpose(y), test_size=0.3,random_state=random_state)

y_score_tests = classifier.fit(X_train, y_train).predict(X_test)
y_score_train=classifier.fit(X_train, y_train).predict(X_train)


In [ ]:

print("-------------------Training Data Evaluation----------------")
print(confusion_matrix(y_train.argmax(axis=1),y_score_train.argmax(axis=1)))
print(classification_report(y_train,y_score_train))
print(accuracy_score(y_train,y_score_train))

In [ ]:
print("---------------/Test Data Evaluation/-------------------------------------")

print(confusion_matrix(y_test.argmax(axis=1),y_score_tests.argmax(axis=1)))

print(classification_report(y_test,y_score_tests))
print(accuracy_score(y_test,y_score_tests))


In [ ]:

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score_tests[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score_tests.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

#ROC curve for a specific class here for the class 2
#print(roc_auc[2])


In [ ]:

plt.figure()
lw = 5
plt.plot(fpr[5], tpr[5], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = it.cycle(['aqua', 'darkorange', 'cornflowerblue','red','green','yellow'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()